In [1]:
from s2h_utils import *
import time
import time

## Load data

In [2]:
data = pd.read_csv("./INPUT/ex2.txt", sep=' ')
data['date_debut'] = pd.to_datetime(data['date_debut'])
data['date_fin'] = pd.to_datetime(data['date_fin'])
data

,id,date_debut,date_fin
0,1,2017-11-20,2017-12-15
1,1,2017-11-20,2017-12-17
2,1,2018-03-20,2018-03-27
3,2,2017-11-20,2017-11-21


In [3]:
dates_presence = data
dates_presence['date'] = dates_presence.apply(lambda x: pd.date_range(x.date_debut, x.date_fin, freq='1d'), axis=1)	
# explode  and drop duplicates: from wide to long data
dates_presence = dates_presence.explode('date')[["id","date"]].drop_duplicates()
dates_presence['presence'] = 1

# GENERATE CALENDAR DATES GIVEN MIN  AND MAX  DATES
baseline = data
baseline = baseline.groupby("id").agg({'date_debut':'min', 'date_fin':'max'}).reset_index()
baseline['date'] = baseline.apply(lambda x: pd.date_range(x.date_debut, x.date_fin, freq='1d'), axis=1)
baseline= baseline.explode('date')[["id","date"]].drop_duplicates()

# JOIN two datasets 
dates_presence = pd.merge(baseline, dates_presence, on =['date', 'id'], how='left')
dates_presence['presence'] = dates_presence['presence'].fillna(0)
dates_presence["year_month"] = dates_presence.date.dt.strftime("%Y-%m")
dates_presence_agg = dates_presence.groupby(["year_month", "id"]).agg({'presence' : 'sum'}).reset_index().sort_values("id")
dates_presence_agg["presence"] = dates_presence_agg["presence"].astype(int)


In [4]:
dates_presence.groupby(["year_month","id"]).sum()

presence
year_month id          
2017-11    1       11.0
           2        2.0
2017-12    1       17.0
2018-01    1        0.0
2018-02    1        0.0
2018-03    1        8.0

In [5]:
        dates_presence["year_month"] = dates_presence.date.dt.strftime("%Y-%m")
        dates_presence_agg = dates_presence.groupby(["year_month", "id"]).agg({'presence' : 'sum'}).reset_index().sort_values("id")
        dates_presence_agg["presence"] = dates_presence_agg["presence"].astype(int)

In [6]:
baseline

,id,date
0,1,2017-11-20
0,1,2017-11-21
0,1,2017-11-22
0,1,2017-11-23
0,1,2017-11-24
...,...,...
0,1,2018-03-25
0,1,2018-03-26
0,1,2018-03-27
1,2,2017-11-20


## REORGANISATION

In [7]:
data = reorganisation_all(data)
data

The algorithm stopped after 2 iterations


,id,date_debut,date_fin
0,1,2017-11-20,2017-12-17
1,1,2018-03-20,2018-03-27
2,2,2017-11-20,2017-11-21


## OUTPUT FINAL

In [8]:
start =time.time()
presence = Presence(data)

In [9]:
output_intermed1, output_intermed2 = presence.process_presence()
# RESULTATS PAR JOUR
output_intermed1.presence.count()

302

In [10]:
output_intermed1

,id,date,presence
0,1,2017-11-01,0.0
1,1,2017-11-02,0.0
2,1,2017-11-03,0.0
3,1,2017-11-04,0.0
4,1,2017-11-05,0.0
...,...,...,...
297,2,2018-03-27,0.0
298,2,2018-03-28,0.0
299,2,2018-03-29,0.0
300,2,2018-03-30,0.0


In [13]:
# RESULTATS PAR MOIS
output_intermed2

,year_month,id,presence
0,2017-11,1,11
2,2017-12,1,17
4,2018-01,1,0
6,2018-02,1,0
8,2018-03,1,8
1,2017-11,2,2
3,2017-12,2,0
5,2018-01,2,0
7,2018-02,2,0
9,2018-03,2,0


In [12]:
output_final = presence.construct_pipeline()
output_final

,id,mois,date_debut_mois,date_fin_mois,NB_arret
0,1,2017-11-01,2017-11-20,2017-11-30,11
1,1,2017-12-01,2017-12-01,2017-12-17,17
2,1,2018-01-01,NaT,NaT,0
3,1,2018-02-01,NaT,NaT,0
4,1,2018-03-01,2018-03-20,2018-03-27,8
5,2,2017-11-01,2017-11-20,2017-11-21,2
6,2,2017-12-01,NaT,NaT,0
7,2,2018-01-01,NaT,NaT,0
8,2,2018-02-01,NaT,NaT,0
9,2,2018-03-01,NaT,NaT,0
